In [12]:
import zipfile, shutil
import sqlite3 as sql
import json, os, sys, datetime
import pprint


def get_temp_dir(path, dir_name='temp'):
    temp_dir = os.path.join(path, dir_name)
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.mkdir(temp_dir)
    return temp_dir


def unpack_apkg(path, unpack_dir=None):
    if not unpack_dir:
        unpack_dir = get_temp_dir(path+'/..', 'unpack')
    db_path = os.path.join(unpack_dir, 'collection.anki2')
    os.mkdir(os.path.join(unpack_dir, 'media'))
    with zipfile.ZipFile(os.path.abspath(path), 'r') as z:
        media = z.read('media').decode()
        media = json.loads(media)
        for i in media:
            fm = z.read(i)
            with open(os.path.join(unpack_dir+'/media', media[i]), 'wb') as f:
                f.write(fm)
        db = z.read('collection.anki2')
        with open(os.path.join(unpack_dir, 'collection.anki2'), 'wb') as f:
            f.write(db)
    db_path = os.path.abspath(db_path)
    with sql.connect(db_path) as dbconn:
        cursor = dbconn.cursor()
        items = cursor.execute('select conf,models,decks,dconf,tags from col').fetchone()
        for i, item_name in enumerate(['conf', 'models', 'decks', 'dconf', 'tags']):
            with open(os.path.join(unpack_dir, item_name+'.json'), 'w') as f:
                f.write(items[i])


unpack_apkg('collection.apkg')
def package_media(path, temp_dir=None, ex={'collection.anki2'}):
    if not temp_dir:
        temp_dir = get_temp_dir(path+'/..')
    media = dict()
    media_files = os.listdir(path)
    media_files = filter(lambda x: x not in ex, media_files)
    for i, m in enumerate(media_files):
        i = str(i)
        media[i] = m
        shutil.copyfile(os.path.join(path, m), os.path.join(temp_dir, i))
    with open(os.path.join(temp_dir, 'media'), 'w') as f:
        json.dump(media, f)


NEW_DB = '''
CREATE TABLE col (
    id              integer primary key,
    crt             integer not null,
    mod             integer not null,
    scm             integer not null,
    ver             integer not null,
    dty             integer not null,
    usn             integer not null,
    ls              integer not null,
    conf            text not null,
    models          text not null,
    decks           text not null,
    dconf           text not null,
    tags            text not null
);
CREATE TABLE notes (
    id              integer primary key,   /* 0 */
    guid            text not null,         /* 1 */
    mid             integer not null,      /* 2 */
    mod             integer not null,      /* 3 */
    usn             integer not null,      /* 4 */
    tags            text not null,         /* 5 */
    flds            text not null,         /* 6 */
    sfld            integer not null,      /* 7 */
    csum            integer not null,      /* 8 */
    flags           integer not null,      /* 9 */
    data            text not null          /* 10 */
);
CREATE TABLE cards (
    id              integer primary key,   /* 0 */
    nid             integer not null,      /* 1 */
    did             integer not null,      /* 2 */
    ord             integer not null,      /* 3 */
    mod             integer not null,      /* 4 */
    usn             integer not null,      /* 5 */
    type            integer not null,      /* 6 */
    queue           integer not null,      /* 7 */
    due             integer not null,      /* 8 */
    ivl             integer not null,      /* 9 */
    factor          integer not null,      /* 10 */
    reps            integer not null,      /* 11 */
    lapses          integer not null,      /* 12 */
    left            integer not null,      /* 13 */
    odue            integer not null,      /* 14 */
    odid            integer not null,      /* 15 */
    flags           integer not null,      /* 16 */
    data            text not null          /* 17 */
);
CREATE TABLE revlog (
    id              integer primary key,
    cid             integer not null,
    usn             integer not null,
    ease            integer not null,
    ivl             integer not null,
    lastIvl         integer not null,
    factor          integer not null,
    time            integer not null,
    type            integer not null
);
CREATE TABLE graves (
    usn             integer not null,
    oid             integer not null,
    type            integer not null
);
CREATE INDEX ix_notes_usn on notes (usn);
CREATE INDEX ix_cards_usn on cards (usn);
CREATE INDEX ix_revlog_usn on revlog (usn);
CREATE INDEX ix_cards_nid on cards (nid);
CREATE INDEX ix_cards_sched on cards (did, queue, due);
CREATE INDEX ix_revlog_cid on revlog (cid);
CREATE INDEX ix_notes_csum on notes (csum);
'''


def new_db(path):
    db_path = os.path.abspath(os.path.join(path, 'collection.anki2'))
    if os.path.exists(db_path):
        os.remove(db_path)
    with sql.connect(db_path) as dbconn:
        cursor = dbconn.cursor()
        cursor.executescript(NEW_DB)

        id = 1
        mod = int(datetime.datetime.now().timestamp()*1000)
        crt = mod // 1000
        # crt = 1470081600
        scm = mod
        ver = 1
        dty = 0
        usn = 0
        ls  = 0
        
        conf = '{}'
        dconf = '{}'
        tags = '{}'
        #############################
        
        

        
        
        
        css = ""
        models = None
        decks = None
        

new_db('.')
def gen_from_apkg(apkg_path, temp_dir=None):
    if not temp_dir:
        temp_dir = get_temp_dir(path+'/..')


package_media('unpack/media')


In [51]:
def unicode_escape(string):
    return string.encode('unicode_escape').decode()

def gen_flds(flds):
    return list([{
                "ord": i,
                "name": name,
                "media": [],
                "sticky": False,
                "rtl": False,
                "font": "Arial",
                "size": 20
            }
            for i, name in enumerate(flds)])

def gen_tmpl(qfmt, afmt, name, desk_id, order):
    return {
        "name": name,
        "qfmt": qfmt,
        "did": desk_id,
        "bafmt": "",
        "afmt": afmt,
        "ord": order,
        "bqfmt": ""
    }

def gen_tmpls(tmpls, desk_id):
    tmpls = list([gen_tmpl(tmpl['qfmt'],
                           tmpl['afmt'],
                           tmpl['name'] if tmpl.get('name') else now_time(),
                           desk_id,
                           order)
                  for order, tmpl in enumerate(tmpls)])
    tmpls[0]['did'] = None
    return tmpls

CSS = '.card {\n font-family: arial;\n font-size: 20px;\n text-align: center;\n color: black;\n background-color: white;\n}\n'

def now_time():
    return str(int(datetime.datetime.now().timestamp()*1000))

def gen_models(model_name, flds, tmpls, css=CSS):
    model_id = now_time()
    desk_id  = now_time()
    
    req = list([[i, "any", [0]] for i in range(len(tmpls))])
    
    flds = gen_flds(flds)
    models = {
            model_id : {
                "vers": [],
                "name": model_name,
                "tags": [],
                "did": desk_id,
                "usn": 0,
                "req": req,
                "flds": flds,
                "sortf": 0,
                "tmpls": gen_tmpls(tmpls, desk_id),
                "mod": model_id[:-3],
                "latexPost": "\\end{document}",
                "type": 0,
                "id": model_id,
                "css": css,
                "latexPre": "\\documentclass[12pt]{article}\n\\special{papersize=3in,5in}\n\\usepackage[utf8]{inputenc}\n\\usepackage{amssymb,amsmath}\n\\pagestyle{empty}\n\\setlength{\\parindent}{0in}\n\\begin{document}\n"
            }
        }
    return models

In [52]:
gen_models('测试', ['名称', '内容', '色彩'], 
           [{'qfmt': '{{名称}}', 'afmt': '{{内容}}是{{色彩}}的', 'name': '卡片 1'},
            {'afmt': '{{名称}}', 'qfmt': '{{内容}}是{{色彩}}的', 'name': '卡片 2'}])

{'1484903489450': {'css': '.card {\n font-family: arial;\n font-size: 20px;\n text-align: center;\n color: black;\n background-color: white;\n}\n',
  'did': '1484903489450',
  'flds': [{'font': 'Arial',
    'media': [],
    'name': '名称',
    'ord': 0,
    'rtl': False,
    'size': 20,
    'sticky': False},
   {'font': 'Arial',
    'media': [],
    'name': '内容',
    'ord': 1,
    'rtl': False,
    'size': 20,
    'sticky': False},
   {'font': 'Arial',
    'media': [],
    'name': '色彩',
    'ord': 2,
    'rtl': False,
    'size': 20,
    'sticky': False}],
  'id': '1484903489450',
  'latexPost': '\\end{document}',
  'latexPre': '\\documentclass[12pt]{article}\n\\special{papersize=3in,5in}\n\\usepackage[utf8]{inputenc}\n\\usepackage{amssymb,amsmath}\n\\pagestyle{empty}\n\\setlength{\\parindent}{0in}\n\\begin{document}\n',
  'mod': '1484903489',
  'name': '测试',
  'req': [[0, 'any', [0]], [1, 'any', [0]]],
  'sortf': 0,
  'tags': [],
  'tmpls': [{'afmt': '{{内容}}是{{色彩}}的',
    'bafmt': '',
 

In [36]:
c = gen_flds(["\u5b57\u6bcd", "\u8bc6\u522b\u7801", "\u4e00\u7ea7\u7b80\u7801", "\u53e3\u8bc0", "\u56fe\u7247", "\u952e\u4f4d"])

In [49]:
with open(r'D:\playground\csv2anki\unpack\models.json', 'r') as f:
    x=json.loads(f.read())
    d= next(iter(x.values()))['tmpls']
    pprint.pprint(d)

[{'afmt': '{{字母}} {{识别码}} {{一级简码}}\n'
          '<hr id=answer>\n'
          '{{口诀}} <br>\n'
          '{{图片}}<br>\n'
          '{{键位}}',
  'bafmt': '',
  'bqfmt': '',
  'did': None,
  'name': '卡片 2',
  'ord': 0,
  'qfmt': '{{字母}} <br>\n{{键位}}'},
 {'afmt': '{{FrontSide}}\n'
          '\n'
          '<hr id=answer>\n'
          '\n'
          '{{字母}} {{识别码}} {{一级简码}} <br>\n'
          '{{图片}}<br>\n'
          '{{键位}}',
  'bafmt': '',
  'bqfmt': '',
  'did': 1470123357010,
  'name': '卡片 3',
  'ord': 1,
  'qfmt': '{{口诀}}'}]


[{'font': 'Arial',
  'media': [],
  'name': '字母',
  'ord': 0,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '识别码',
  'ord': 1,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '一级简码',
  'ord': 2,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '口诀',
  'ord': 3,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '图片',
  'ord': 4,
  'rtl': False,
  'size': 20,
  'sticky': False},
 {'font': 'Arial',
  'media': [],
  'name': '键位',
  'ord': 5,
  'rtl': False,
  'size': 20,
  'sticky': False}]

In [ ]:
cursor = dbconn.cursor

In [2]:
bytes(r"\u53e3\u8bc0", 'utf8').decode('unicode_escape')

'口诀'

In [9]:
cd d:\playground\csv2anki

d:\playground\csv2anki
